# Hydrologic attributes

Notebook to create the file `CAMELS_DE_hydrologic_attributes.csv`.

columns:
- gauge_id
- q_mean [mm/d]
- runoff_ratio [-]
- flow_period_start
- flow_period_end
- flow_perc_complete [-]
- slope_fdc [-]
- hfd_mean [days]
- Q5 [mm/d]
- Q95 [mm/d]
- high_q_freq [days/year]
- high_q_dur [days]
- low_q_freq [days/year]
- low_q_dur [days]
- zero_q_freq [-]

In [1]:
from glob import glob
import pandas as pd
import numpy as np

In [2]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 5


## Create the file `CAMELS_DE_hydrologic_attributes.csv`

In [3]:
def filter_complete_hydro_years(df, tolerance=0.05):
    """
    Helper function to filter a DataFrame to only include complete hydrological 
    years (October - September). A hydrological year is considered complete if 
    it has less than the specified tolerance of missing values.

    """
    # if date is not in index, set it as index
    if 'date' in df.columns:
        df = df.set_index('date')

    # convert the index to datetime if it is not already
    if not isinstance(df.index, pd.DatetimeIndex): 
        df.index = pd.to_datetime(df.index)

    # make the dataframe start at the beginning of the hydrological year, i.e. 01.10. of the previous year
    min_year = df.index.year.min()
    df = df.reindex(pd.date_range(start=f"{min_year-1}-10-01", end=df.index.max(), freq='D'))

    # make the dataframe end at the end of the hydrological year, i.e. 30.09. of the next year
    max_year = df.index.year.max()
    df = df.reindex(pd.date_range(start=df.index.min(), end=f"{max_year+1}-09-30", freq='D'))

    # Calculate the number of missing values per hydrological year for 'discharge_vol' column
    df['hydro_year'] = df.index.year
    df.loc[df.index.month >= 10, 'hydro_year'] += 1
    missing_values_per_year = df['discharge_vol'].groupby(df['hydro_year']).apply(lambda x: x.isna().mean())

    # Filter the DataFrame to only include years with less than the tolerance of missing values
    df_filtered = df[df['hydro_year'].isin(missing_values_per_year[missing_values_per_year <= tolerance].index)]

    # Drop the 'hydro_year' column
    df_filtered = df_filtered.drop(columns='hydro_year')

    return df_filtered

### `q_mean` and `runoff_ratio`

In [4]:
# dataframe to store results
df_results = pd.DataFrame(index=camels_ids)

for camels_id in camels_ids:
    # get station data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv", index_col=0)

    # filter complete hydro years
    df = filter_complete_hydro_years(df, tolerance=0.05)

    # q_mean and p_mean
    q_mean = df["discharge_spec"].mean()
    p_mean = df["precipitation_mean"].mean()

    # q_mean
    df_results.loc[camels_id, "q_mean"] = round(q_mean, 2)

    # runoff_ratio
    df_results.loc[camels_id, "runoff_ratio"] = round(q_mean / p_mean, 2)

df_results

,q_mean,runoff_ratio
1,1.45,0.43
2,0.86,0.24
3,1.91,0.49
4,1.12,0.32
5,0.48,0.15


In [5]:
df_results[df_results["runoff_ratio"] > 1]

,q_mean,runoff_ratio


### `flow_period_start`, `flow_period_end`, `flow_perc_complete`

We also include the columns `flow_period_start`, `flow_period_end`, and `flow_perc_complete` from the file `CAMELS_GB_hydrometry_attributs.csv`, as we do not include uncertainty hydrometry attributes.

**Here, we do not filter for complete hydrological years!**

In [6]:
for camels_id in camels_ids:
    # get station data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv", index_col=0)

    # parse dates
    df.index = pd.to_datetime(df.index)

    # flow_period_start: first date in the timeseries where discharge_vol is not NaN
    flow_period_start = df.index[df["discharge_vol"].notnull()].min()
    df_results.loc[camels_id, "flow_period_start"] = flow_period_start

    # flow_period_end
    flow_period_end = df.index[df["discharge_vol"].notnull()].max()
    df_results.loc[camels_id, "flow_period_end"] = flow_period_end

    # flow_perc_complete: percentage of days with discharge_vol not NaN
    flow_perc_complete = sum(df["discharge_vol"].notnull()) / len(df)
    df_results.loc[camels_id, "flow_perc_complete"] = round(flow_perc_complete, 5) * 100 # round to 5 decimal places for better precision here

df_results

,q_mean,runoff_ratio,flow_period_start,flow_period_end,flow_perc_complete
1,1.45,0.43,1963-11-01,2020-12-31,81.668
2,0.86,0.24,1951-01-01,2020-12-31,100.000
3,1.91,0.49,1951-01-01,2020-12-31,82.619
4,1.12,0.32,1951-01-01,2020-12-31,100.000
5,0.48,0.15,1979-11-01,2020-12-31,58.812


### `stream_elas`

*streamflow precipitation elasticity (sensitivity of streamflow to changes in precipitation at the annual timescale, using the mean daily discharge as reference)*

# https://github.com/camels-ch/camels/blob/master/hydro_climate_attributes/hydro_signatures.R


**machen wir nicht**

### `slope_fdc`

*slope of the flow duration curve (between the log-transformed 33rd and 66th streamflow percentiles)*

In [7]:
for camels_id in camels_ids:
    # get station data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # make a new df with only the "discharge_spec" column
    df_fdc = df[["discharge_spec"]]

    # drop NaN values
    df_fdc = df_fdc.dropna()

    # drop 0 values
    df_fdc = df_fdc[df_fdc["discharge_spec"] > 0]

    # calculate the 33rd and 66th percentile
    p33 = df_fdc["discharge_spec"].quantile(0.33)
    p66 = df_fdc["discharge_spec"].quantile(0.66)

    # calculate the slope between the log-transformed 33rd and 66th percentiles
    slope = (np.log(p66) - np.log(p33)) / (0.66 - 0.33)

    # add to results
    df_results.loc[camels_id, "slope_fdc"] = round(slope, 2)
    

df_results

,q_mean,runoff_ratio,flow_period_start,flow_period_end,flow_perc_complete,slope_fdc
1,1.45,0.43,1963-11-01,2020-12-31,81.668,2.80
2,0.86,0.24,1951-01-01,2020-12-31,100.000,2.93
3,1.91,0.49,1951-01-01,2020-12-31,82.619,2.22
4,1.12,0.32,1951-01-01,2020-12-31,100.000,3.07
5,0.48,0.15,1979-11-01,2020-12-31,58.812,3.12


### `hfd_mean`

*mean half-flow date (date on which the cumulative discharge since 1 October reaches half of the annual discharge) [days since 1st October]*

In [8]:
for camels_id in camels_ids:
    # get station data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv", index_col=0)

    # filter complete hydro years
    df = filter_complete_hydro_years(df, tolerance=0.05)

    # add column for the water year
    df['water_year'] = df.index.year
    df.loc[df.index.month < 10, 'water_year'] -= 1

    # calculate annual and half discharge
    annual_discharge = df.groupby('water_year')['discharge_vol'].sum()
    half_discharge = annual_discharge / 2

    # calculate the cumulative discharge for each water year
    df['cumulative_discharge'] = df.groupby('water_year')['discharge_vol'].cumsum()

    # for each water year, find the date where the cumulative discharge first exceeds the half total discharge
    df['exceeds_half'] = df.groupby('water_year')['cumulative_discharge'].transform(lambda x: x > half_discharge[x.name])
    half_flow_dates = df[df['exceeds_half']].groupby('water_year')['exceeds_half'].idxmax()

    # create a new DatetimeIndex representing October 1st of the water year
    october_1st = pd.to_datetime(half_flow_dates.dt.year.astype(str) + '-10-01')
    october_1st[half_flow_dates.dt.month < 10] = pd.to_datetime((half_flow_dates[half_flow_dates.dt.month < 10].dt.year - 1).astype(str) + '-10-01')

    # subtract October 1st from each of these dates to get the number of days since October 1st
    # add one day to include the end date in the calculation
    days_since_october = (half_flow_dates - october_1st).dt.days + 1

    # calculate the mean of these values
    hfd_mean = days_since_october.mean()

    # add to results
    df_results.loc[camels_id, "hfd_mean"] = round(hfd_mean, 2)

df_results

,q_mean,runoff_ratio,flow_period_start,flow_period_end,flow_perc_complete,slope_fdc,hfd_mean
1,1.45,0.43,1963-11-01,2020-12-31,81.668,2.80,149.98
2,0.86,0.24,1951-01-01,2020-12-31,100.000,2.93,147.71
3,1.91,0.49,1951-01-01,2020-12-31,82.619,2.22,163.60
4,1.12,0.32,1951-01-01,2020-12-31,100.000,3.07,148.38
5,0.48,0.15,1979-11-01,2020-12-31,58.812,3.12,148.60


### `Q5` and `Q95`

- *5% flow quantile (low flow) [mm/day]*
- *95% flow quantile (high flow)[mm/day]*


In [9]:
for camels_id in camels_ids:
    # get station data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv", index_col=0)

    # filter complete hydro years
    df = filter_complete_hydro_years(df, tolerance=0.05)

    # discharge 5 and 95 percentile
    q5 = df["discharge_spec"].quantile(0.05)
    q95 = df["discharge_spec"].quantile(0.95)

    # add to results
    df_results.loc[camels_id, "Q5"] = round(q5, 2)
    df_results.loc[camels_id, "Q95"] = round(q95, 2)

df_results

,q_mean,runoff_ratio,flow_period_start,flow_period_end,flow_perc_complete,slope_fdc,hfd_mean,Q5,Q95
1,1.45,0.43,1963-11-01,2020-12-31,81.668,2.80,149.98,0.15,4.74
2,0.86,0.24,1951-01-01,2020-12-31,100.000,2.93,147.71,0.07,3.04
3,1.91,0.49,1951-01-01,2020-12-31,82.619,2.22,163.60,0.31,6.52
4,1.12,0.32,1951-01-01,2020-12-31,100.000,3.07,148.38,0.12,3.99
5,0.48,0.15,1979-11-01,2020-12-31,58.812,3.12,148.60,0.04,1.60


### `high_q_freq`, `high_q_dur`, `low_q_freq`, `low_q_dur` and `zero_q_freq`

- *frequency of high-flow days (> 9 times the median daily flow) [days/yr]*
- *average duration of high flow events (number of consecutive days >9 times the median daily flow) [days]*
- *frequency of low flow days (< 0.2 times the mean daily flow) [days/yr]*
- *average duration of low flow events (number of consecutive days < 0.2 times the mean daily flow) [days]*
- *frequency of days with Q = 0 [-]*

In [10]:
for camels_id in camels_ids:
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # filter complete hydro years
    df = filter_complete_hydro_years(df, tolerance=0.05)

    # Time steps considered as high flows
    hf = df["discharge_spec"] > 9 * df["discharge_spec"].median()

    if hf.any():
        # Mean duration of daily high flow events
        # a string where 1 indicates a high flow event
        hf_bin = ''.join(hf.astype(int).astype(str))
        # Use str.split to isolate successive time steps with high discharge
        hf_dur_noise = [len(s) for s in hf_bin.split('0')]
        # Mean duration
        high_q_dur = np.mean([d for d in hf_dur_noise if d > 0])

        # Average number of daily high-flow events per year
        high_q_freq = hf.sum() / len(df) * 365.25
    else:
        high_q_dur = 0
        high_q_freq = 0

    # Time steps considered as low flows
    lf = df["discharge_spec"] < 0.2 * df["discharge_spec"].mean()

    if lf.any():
        # Mean duration of daily low flow events
        # a string where 1 indicates a low flow event
        lf_bin = ''.join(lf.astype(int).astype(str))
        # Use str.split to isolate successive time steps with low discharge
        lf_dur_noise = [len(s) for s in lf_bin.split('0')]
        # Mean duration
        low_q_dur = np.mean([d for d in lf_dur_noise if d > 0])

        # Average number of daily low-flow events per year
        low_q_freq = lf.sum() / len(df) * 365.25

    # Ratio of time steps with zero discharge
    zero_q_freq = df["discharge_spec"].eq(0).mean()

    # add to results
    df_results.loc[camels_id, "high_q_freq"] = round(high_q_freq, 2)
    df_results.loc[camels_id, "high_q_dur"] = round(high_q_dur, 2)
    df_results.loc[camels_id, "low_q_freq"] = round(low_q_freq, 2)
    df_results.loc[camels_id, "low_q_dur"] = round(low_q_dur, 2)
    df_results.loc[camels_id, "zero_q_freq"] = round(zero_q_freq, 2)

df_results

,q_mean,runoff_ratio,flow_period_start,flow_period_end,flow_perc_complete,slope_fdc,hfd_mean,Q5,Q95,high_q_freq,high_q_dur,low_q_freq,low_q_dur,zero_q_freq
1,1.45,0.43,1963-11-01,2020-12-31,81.668,2.80,149.98,0.15,4.74,7.48,3.04,62.05,9.07,0.0
2,0.86,0.24,1951-01-01,2020-12-31,100.000,2.93,147.71,0.07,3.04,10.51,2.21,74.39,8.82,0.0
3,1.91,0.49,1951-01-01,2020-12-31,82.619,2.22,163.60,0.31,6.52,9.20,1.61,37.36,10.28,0.0
4,1.12,0.32,1951-01-01,2020-12-31,100.000,3.07,148.38,0.12,3.99,12.35,2.65,67.01,9.90,0.0
5,0.48,0.15,1979-11-01,2020-12-31,58.812,3.12,148.60,0.04,1.60,7.98,2.98,54.12,8.95,0.0


## Save results

In [11]:
# save results
df_results.to_csv("../output_data/camels_de/CAMELS_DE_hydrologic_attributes.csv", index_label="gauge_id")